In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ab_data = pd.read_csv('data/ab_data.csv')

In [ ]:
ab_data.head()

In [ ]:
ab_data.info()

In [ ]:
ab_data['timestamp'] = pd.to_datetime(ab_data['timestamp'], format='%Y-%m-%d')

In [ ]:
daily_data = ab_data.groupby(['timestamp','group']).agg({
    'user_id':'count',
    'converted':'sum'
}).reset_index().rename(columns={'user_id': 'users_count'})
daily_data.head()

In [ ]:
daily_data['conversion'] = (daily_data['converted']/daily_data['users_count']) * 100

daily_data[daily_data['timestamp'] == '2017-01-05'] 

In [ ]:
# создаём фигуру размером 8x4
fig = plt.figure(figsize=(8, 4)) 
# добавляем систему координат
ax = fig.add_axes([1, 1, 1, 1]) 
# строим boxplot для conversion по признаку group
sns.boxplot(data=daily_data, x='conversion', y='group', ax=ax) 
# задаём подпись к графику
ax.set_title('Сравнение конверсии в группах')
# задаём отображение сетки
ax.grid();

In [ ]:
conversion_piv = daily_data.groupby('group')['conversion'].agg(
    ['mean', 'median']
)
display(conversion_piv)

In [ ]:
# создаём фигуру размером 8x4
fig = plt.figure(figsize=(8, 4))
# добавляем систему координат
ax = fig.add_axes([1, 1, 1, 1])
# строим lineplot для конверсии во времени в каждой группе
sns.lineplot(
    data=daily_data,
    x='timestamp',
    y='conversion',
    hue='group', 
    ax=ax
)
# задаём подпись к графику
ax.set_title('График конверсии по дням')
# задаём поворот меток на оси абсцисс
ax.xaxis.set_tick_params(rotation=45)
# задаём отображение сетки
ax.grid();

In [ ]:
# выделяем данные группы А
daily_data_a = daily_data[daily_data['group'] == 'A']
# считаем кумулятивное количество посетителей
daily_data_a['cum_users_count'] = daily_data_a['users_count'].cumsum()
# выводим время, количество посетителей и кумулятивное количество посетителей
display(daily_data_a[['timestamp', 'users_count', 'cum_users_count']].head())

In [ ]:
# вычисляем кумулятивную сумму количества посетителей
daily_data['cum_users_count'] = daily_data.groupby(['group'])['users_count'].cumsum()
# вычисляем кумулятивную сумму количества совершённых целевых действий
daily_data['cum_converted'] = daily_data.groupby(['group'])['converted'].cumsum()
# вычисляем кумулятивную конверсию
daily_data['cum_conversion'] = daily_data['cum_converted']/daily_data['cum_users_count'] * 100
display(daily_data.head())

In [ ]:
# создаём фигуру размером 8x4
fig = plt.figure(figsize=(8, 4))
# добавляем систему координат
ax = fig.add_axes([1, 1, 1, 1])
# строим lineplot для кумулятивной конверсии во времени в каждой группе
sns.lineplot(x='timestamp', y='cum_conversion', data=daily_data, hue='group', ax=ax)
# задаём подпись к графику
ax.set_title('График кумулятивной конверсии по дням')
# задаём поворот меток на оси абсцисс
ax.xaxis.set_tick_params(rotation = 45)
# задаём отображение сетки
ax.grid(True);

In [ ]:
sample_a = pd.read_csv('data/ab_test-redesign_sample_a.csv')
sample_b = pd.read_csv('data/ab_test-redesign_sample_b.csv')

In [ ]:
sample_ab = pd.concat([sample_a,sample_b])

In [ ]:
sample_ab.head()

In [ ]:
sample_ab['date'] = pd.to_datetime(sample_ab['date'], format='%Y-%m-%d')
sample_ab = sample_ab.sort_values(by='date', ignore_index=True)


In [ ]:
sample_ab_grouped =  sample_ab.groupby(['date','group']).agg({
    'cid':'count',
    'transactions':'sum',
    'revenue':'sum'
}).reset_index().rename(columns={'transactions': 'transactions_sum', 'cid':'cid_count'})

sample_ab_grouped.head()

In [ ]:
sample_ab_cum = sample_ab_grouped.copy()

sample_ab_cum['cum_cid_count'] = sample_ab_cum.groupby(['group'])['cid_count'].cumsum()
sample_ab_cum['cum_transactions_sum'] = sample_ab_cum.groupby(['group'])['transactions_sum'].cumsum()
sample_ab_cum['cum_revenue'] = sample_ab_cum.groupby(['group'])['revenue'].cumsum()

sample_ab_cum['conversion'] = sample_ab_cum['cid_count']/sample_ab_cum['transactions_sum']
sample_ab_cum['mean_cheak'] = sample_ab_cum['revenue']/sample_ab_cum['transactions_sum']

sample_ab_cum['cum_conversion'] = sample_ab_cum['cum_transactions_sum']/sample_ab_cum['cum_cid_count'] * 100
sample_ab_cum['cum_mean_cheak'] = sample_ab_cum['cum_revenue']/sample_ab_cum['cum_transactions_sum']


sample_ab_cum.tail()

In [ ]:
fig, axes = plt.subplots(figsize=(10, 5))
sns.lineplot(x='date', y='cum_conversion', data=sample_ab_cum, hue='group')
axes.set_title('График кумулятивной конверсии по дням')
# задаём поворот меток на оси абсцисс
axes.xaxis.set_tick_params(rotation = 45)
# задаём отображение сетки
axes.grid(True)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 5))
sns.lineplot(x='date', y='cum_mean_cheak', data=sample_ab_cum, hue='group')
axes.set_title('График кумулятивной конверсии по дням')
# задаём поворот меток на оси абсцисс
axes.xaxis.set_tick_params(rotation = 45)
# задаём отображение сетки
axes.grid(True)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
converted_piv= ab_data.groupby('group')['converted'].agg(
    ['sum', 'count']
)
display(converted_piv)

In [ ]:
# нулевая и альтернативная гипотезы
H0 = 'Конверсии в группах А и B равны'
H1 = 'Конверсия в группе А выше, чем конверсия в группе B'
alpha = 0.05 # уровень значимости
# вычисляем значение p-value для z-теста для пропорций
_, p_value = proportions_ztest(
    count=converted_piv['sum'], #число "успехов"
    nobs=converted_piv['count'], #общее число наблюдений
    alternative='larger',
)
# выводим результат на экран
print('p-value: ', round(p_value, 2))
# сравниваем полученное p-value с уровнем значимости
if (p_value < alpha): 
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))
 
# p-value:  0.1
# Принимаем нулевую гипотезу. Конверсии в группах А и B равны

In [ ]:
time_data = pd.read_excel('data/time_data.xlsx')

In [ ]:
time_data.info()

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (12,4))
sns.histplot(x='time(A)', data = time_data, ax = axes[0])
sns.histplot(x='time(B)', data = time_data, ax = axes[1])

axes[0].set_title('Распределение времени на сайте по группе A');
axes[1].set_title('Распределение времени на сайте по группе B');

In [ ]:
from scipy.stats import shapiro

H0 = 'Распределения в обеих группах нормальные'
H1 = 'Распределение в одной или обеих группах отлично от нормального'
alpha = 0.05 # уровень значимости

shapiro_result_a = shapiro(time_data['time(A)'])
shapiro_result_b = shapiro(time_data['time(B)'])

print('p-value группы А', round(shapiro_result_a.pvalue, 2))
print('p-value группы B', round(shapiro_result_b.pvalue, 2))

if shapiro_result_a.pvalue < alpha or shapiro_result_b.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))

In [ ]:
from scipy.stats import ttest_ind
# нулевая и альтернативная гипотезы
H0 = 'Среднее время в группах одинаково'
H1 = 'Среднее время в группе А меньше, чем в группе B'
alpha = 0.05 #уровень значимости
#вычисляем результат T-теста для выборок
results = ttest_ind(
    a=time_data['time(A)'],
    b=time_data['time(B)'], 
    alternative='less'
)
print('p-value:', round(results.pvalue, 2))
 
if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))

In [ ]:
sample_ab_cum

In [ ]:
sample_ab.head()

In [ ]:
from  statsmodels.stats.proportion import proportions_ztest

converted_piv= sample_ab.groupby('group')['transactions'].agg(
    ['sum', 'count']
)
display(converted_piv)

# нулевая и альтернативная гипотезы
H0 = 'Конверсии в группах А и B равны'
H1 = 'Конверсия в группе А ниже, чем конверсия в группе B'
alpha = 0.05 # уровень значимости
# вычисляем значение p-value для z-теста для пропорций
_, p_value = proportions_ztest(
    count=converted_piv['sum'], #число "успехов"
    nobs=converted_piv['count'], #общее число наблюдений
    alternative='smaller'
)
# выводим результат на экран
print('p-value: ', round(p_value, 2))
# сравниваем полученное p-value с уровнем значимости
if (p_value < alpha): 
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))
 
# p-value:  0.1
# Принимаем нулевую гипотезу. Конверсии в группах А и B равны
 
 

In [ ]:
sample_ab_cum.head()

In [ ]:
from scipy.stats import shapiro

sample_ab_cum['mean_cheak']

H0 = 'Распределения в обеих группах нормальные'
H1 = 'Распределение в одной или обеих группах отлично от нормального'
alpha = 0.05 # уровень значимости

shapiro_result_a = shapiro(sample_ab_cum[sample_ab_cum['group'] == 'A']['mean_cheak'])
shapiro_result_b = shapiro(sample_ab_cum[sample_ab_cum['group'] == 'B']['mean_cheak'])

print('p-value группы А', round(shapiro_result_a.pvalue, 2))
print('p-value группы B', round(shapiro_result_b.pvalue, 2))

if shapiro_result_a.pvalue < alpha or shapiro_result_b.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))

In [ ]:
from scipy.stats import mannwhitneyu

H0 = 'Ежедневный средний чек равен'
H1 = 'Ежидневный средний чек в группе A меньше чем в группе B'
alpha = 0.1 # уровень значимости

result_a = sample_ab_cum[sample_ab_cum['group'] == 'A']['mean_cheak']
result_b = sample_ab_cum[sample_ab_cum['group'] == 'B']['mean_cheak']

__, p = mannwhitneyu(result_a, result_b, alternative = 'less')
print(p)

if p < alpha:
    print("Отвергаем нулевую гипотезу: {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))


In [ ]:
from scipy.stats import norm
z_crit = -norm.ppf(alpha/2) # z критическое


n = 36 # размер выборки
x_mean = 16100 # выборочное среднее
sigma = 12000 # истинное стандартное отклонение
gamma = 0.95 # уровень надёжности
alpha = 1 - gamma # уровень значимости



eps = z_crit * sigma/(n ** 0.5) #погрешность
lower_bound = x_mean - eps # левая (нижняя) граница
upper_bound = x_mean + eps # правая (верхняя) граница
confidence_interval = (round(lower_bound), round(upper_bound)) # создаём кортеж из округлённых границ интервала
print('Доверительный интервал: {}'.format(confidence_interval)) # выводим результат
 
 
 
# Доверительный интервал: (12180, 20020)

In [56]:
n = 64 # размер выборки
x_mean =12.35 # выборочное среднее
sigma = 2.4 # истинное стандартное отклонение
gamma = 0.90 # уровень надёжности
alpha = 1 - gamma # уровень значимости

z_crit = -norm.ppf(alpha/2)
eps = z_crit * sigma/(n ** 0.5) #погрешность
lower_bound = x_mean - eps # левая (нижняя) граница
upper_bound = x_mean + eps # правая (верхняя) граница
confidence_interval = (round(lower_bound,2), round(upper_bound,2)) # создаём кортеж из округлённых границ интервала
print(z_crit)
print('Доверительный интервал: {}'.format(confidence_interval)) # выводим результат
 

1.6448536269514729
Доверительный интервал: (11.86, 12.84)


In [ ]:
from scipy.stats import t

n = 15 # размер выборки
k = n - 1 # число степеней свободы
x_mean = 2000 # выборочное среднее
x_std = 400 # выборочное стандартное отклонение
gamma = 0.95 # уровень надёжности
alpha = 1 - gamma # уровень значимости


t_crit = -t.ppf(alpha/2, k) # t-критическое


eps = t_crit * x_std/(n ** 0.5) # погрешность
lower_bound = x_mean - eps # левая (нижняя) граница
upper_bound = x_mean + eps # правая (верхняя) граница
confidence_interval = (round(lower_bound), round(upper_bound)) # создаём кортеж из округлённых границ интервала
print('Доверительный интервал: {}'.format(confidence_interval)) # выводим результат
 
 
# Доверительный интервал: (1778, 2222)

In [58]:
from scipy.stats import t

n = 25 # размер выборки
k = n - 1 # число степеней свободы
x_mean = 3540 # выборочное среднее
x_std = 1150 # выборочное стандартное отклонение
gamma = 0.90 # уровень надёжности
alpha = 1 - gamma # уровень значимости


t_crit = -t.ppf(alpha/2, k) # t-критическое


eps = t_crit * x_std/(n ** 0.5) # погрешность
lower_bound = x_mean - eps # левая (нижняя) граница
upper_bound = x_mean + eps # правая (верхняя) граница
confidence_interval = (round(lower_bound), round(upper_bound)) # создаём кортеж из округлённых границ интервала
print('Доверительный интервал: {}'.format(confidence_interval)) # выводим результат
 
 
# Доверительный интервал: (1778, 2222)

Доверительный интервал: (3146, 3934)
